In [7]:
#from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

In [8]:
import pandas as pd
import numpy as np
import string, os 

In [9]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
os.listdir(curr_dir)

['ArticlesApril2017.csv',
 'ArticlesApril2018.csv',
 'ArticlesFeb2017.csv',
 'ArticlesFeb2018.csv',
 'ArticlesJan2017.csv',
 'ArticlesJan2018.csv',
 'ArticlesMarch2017.csv',
 'ArticlesMarch2018.csv',
 'ArticlesMay2017.csv']

In [12]:
curr_dir = 'input/'
all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(curr_dir + filename)
        all_headlines.extend(list(article_df.headline.values))
    break

all_headlines = [h for h in all_headlines if h != "Unknown"] # removes unknown headlines
print(all_headlines)
len(all_headlines)

['Finding an Expansive View  of a Forgotten People in Niger', 'And Now,  the Dreaded Trump Curse', 'Venezuela’s Descent Into Dictatorship', 'Stain Permeates Basketball Blue Blood', 'Taking Things for Granted', 'The Caged Beast Awakens', 'An Ever-Unfolding Story', 'O’Reilly Thrives as Settlements Add Up', 'Mouse Infestation', 'Divide in G.O.P. Now Threatens Trump Tax Plan', 'Variety Puzzle: Acrostic', 'They Can Hit a Ball 400 Feet. But Play Catch? That’s Tricky.', 'In Trump Country, Shock at Trump Budget Cuts', 'Why Is This Hate Different From All Other Hate?', 'Pick Your Favorite Ethical Offender', 'My Son’s Growing Black Pride', 'Jerks and the Start-Ups They Ruin', 'Trump  Needs  a Brain', 'Manhood in the Age of Trump', 'The Value of a Black College', 'Initial Description', 'Rough Estimates', 'El Pasatiempo Nacional', 'Cooling Off on a Hot Day at Yankee Stadium', 'Trump’s Staff Mixed Politics and Paydays', 'A Virtuoso Rebuilding Act Requires Everyone in Tune', '‘Homeland,’ Season 6, E

831

In [13]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]
# cleans all punctuations from given string

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted',
 'the caged beast awakens',
 'an everunfolding story',
 'oreilly thrives as settlements add up',
 'mouse infestation',
 'divide in gop now threatens trump tax plan']

In [14]:
#Generating Sequence of N-gram Tokens

In [16]:
t  = Tokenizer()
fit_text = "The earth is an awesome place live"
#print(dir(t))
t.fit_on_texts(fit_text)
#print("word_index:",t.word_index)
test_text = "The earth is an great place live"
sequences = t.texts_to_sequences(test_text)
test_sequences = []
for i in test_text:
    
print("sequences : ",sequences,'\n')

print("word_index : ",t.word_index)

IndentationError: expected an indented block (Temp/ipykernel_19708/971059913.py, line 11)

In [ ]:
# --> https://stackoverflow.com/questions/51956000/what-does-keras-tokenizer-method-exactly-do
# this link gives you a exact solution

In [ ]:
# Tokenizer --> if you pass a list to tokenizer word_index is going to be words but if you pass string then the
# word_index will be letters


In [17]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        print("printed line :   ",tokenizer.texts_to_sequences([line])[0])
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

printed line :    [169, 17, 665, 367, 4, 2, 666, 170, 5, 667]
printed line :    [6, 80, 1, 668, 10, 669]
printed line :    [670, 671, 129, 672]
printed line :    [673, 674, 368, 675, 676]
printed line :    [105, 171, 8, 677]
printed line :    [1, 678, 679, 680]
printed line :    [17, 681, 227]
printed line :    [130, 682, 22, 683, 369, 33]
printed line :    [684, 685]
printed line :    [228, 5, 229, 80, 686, 10, 55, 51]
printed line :    [131, 172, 230]
printed line :    [81, 43, 687, 2, 688, 689, 690, 27, 691, 692, 231, 693]
printed line :    [5, 10, 694, 370, 12, 10, 371, 372]
printed line :    [38, 9, 25, 106, 107, 18, 39, 232, 106]
printed line :    [233, 44, 695, 696, 373]
printed line :    [40, 374, 697, 132, 698]
printed line :    [699, 6, 1, 700, 81, 701]
printed line :    [10, 702, 2, 133]
printed line :    [703, 5, 1, 375, 4, 10]
printed line :    [1, 376, 4, 2, 132, 68]
printed line :    [704, 705]
printed line :    [706, 707]
printed line :    [708, 709, 710]
printed line :

[[169, 17],
 [169, 17, 665],
 [169, 17, 665, 367],
 [169, 17, 665, 367, 4],
 [169, 17, 665, 367, 4, 2],
 [169, 17, 665, 367, 4, 2, 666],
 [169, 17, 665, 367, 4, 2, 666, 170],
 [169, 17, 665, 367, 4, 2, 666, 170, 5],
 [169, 17, 665, 367, 4, 2, 666, 170, 5, 667],
 [6, 80]]

In [ ]:
inp_sequences[:10]

In [ ]:
tokenizer.word_index

In [ ]:
tokenizer.texts_to_sequences('Finding an Expansive View  of a Forgotten People in Niger')[1]

In [18]:
max_inp_sequences = [len(x) for x in inp_sequences]
max_inp_sequences

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 2,
 3,
 4,
 5,
 6,
 2,
 3,
 4,
 2,
 3,
 4,
 5,
 2,
 3,
 4,
 2,
 3,
 4,
 2,
 3,
 2,
 3,
 4,
 5,
 6,
 2,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 2,
 3,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 2,
 3,
 4,
 5,
 2,
 3,
 4,
 5,
 2,
 3,
 4,
 5,
 6,
 2,
 3,
 4,
 2,
 3,
 4,
 5,
 6,
 2,
 3,
 4,
 5,
 6,
 2,
 2,
 2,
 3,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 2,
 3,
 4,
 5,
 6,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 2,
 3,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 2,
 3,
 4,
 5,
 6,
 2,
 3,
 4,
 5,
 6,
 2,
 3,
 4,
 5,
 2,
 3,
 4,
 2,
 3,
 4,
 2,
 3,
 4,
 5,
 6,
 7,
 2,
 3,
 4,
 5,
 2,
 3,
 4,
 5,
 2,
 3,
 4,
 5,
 6,
 2,
 3,
 4,
 5,
 6,
 2,
 3,
 4,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 2,
 3,

In [19]:
import tensorflow as tf

In [ ]:
max([len(x) for x in inp_sequences])


In [ ]:
'''
Now that we have generated a data-set which contains sequence of tokens, it is possible that different sequences have different lengths. 
Before starting training the model, we need to pad the sequences and make their lengths equal.
'''

In [ ]:
inp_sequences

In [ ]:
tf.keras.preprocessing.sequence.pad_sequences(inp_sequences)[:,:-1]

In [ ]:
tf.keras.preprocessing.sequence.pad_sequences(inp_sequences)

In [ ]:
inp_sequences[:-1]

In [ ]:
myNestedList = np.array([[1, 2], [3, 4]])
myNestedList[:,-1]

In [ ]:
myNestedList[:,:-1]

In [20]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences]) #for padding
    # we just make sure that every n gram token has the fixed same size
    input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [ ]:
inp_sequences[:10]

In [ ]:
predictors

In [ ]:
'''
LSTM -> Unlike Feed-forward neural networks in which activation outputs are propagated only in one direction, the activation outputs from neurons propagate in both directions (from inputs to outputs and from outputs to inputs) in Recurrent Neural Networks. 
This creates loops in the neural network architecture which acts as a ‘memory state’ of the neurons. 
This state allows the neurons an ability to remember what have been learned so far.


The memory state in RNNs gives an advantage over traditional neural networks but a problem called Vanishing Gradient is associated with them. 
In this problem, while learning with a large number of layers, it becomes really hard for the network to learn and tune the parameters of the earlier layers.
To address this problem, A new type of RNNs called LSTMs (Long Short Term Memory) Models have been developed.
'''

Input Layer : Takes the sequence of words as input
LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later.
Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer. It helps in preventing over fitting. (Optional Layer)
Output Layer : Computes the probability of the best possible next word as output

In [92]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.2))

    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 18, 10)            24220     
                                                                 
 lstm_3 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 2422)              244622    
                                                                 
Total params: 313,242
Trainable params: 313,242
Non-trainable params: 0
_________________________________________________________________


In [93]:
model.fit(predictors, label, epochs=100, verbose=2)

Epoch 1/100
151/151 - 3s - loss: 7.3773 - 3s/epoch - 20ms/step
Epoch 2/100
151/151 - 2s - loss: 6.9063 - 2s/epoch - 10ms/step
Epoch 3/100
151/151 - 2s - loss: 6.7994 - 2s/epoch - 10ms/step
Epoch 4/100
151/151 - 2s - loss: 6.7199 - 2s/epoch - 10ms/step
Epoch 5/100
151/151 - 2s - loss: 6.6517 - 2s/epoch - 10ms/step
Epoch 6/100
151/151 - 2s - loss: 6.5834 - 2s/epoch - 10ms/step
Epoch 7/100
151/151 - 2s - loss: 6.5030 - 2s/epoch - 10ms/step
Epoch 8/100
151/151 - 2s - loss: 6.4174 - 2s/epoch - 10ms/step
Epoch 9/100
151/151 - 2s - loss: 6.3313 - 2s/epoch - 10ms/step
Epoch 10/100
151/151 - 2s - loss: 6.2378 - 2s/epoch - 10ms/step
Epoch 11/100
151/151 - 2s - loss: 6.1549 - 2s/epoch - 10ms/step
Epoch 12/100
151/151 - 2s - loss: 6.0702 - 2s/epoch - 10ms/step
Epoch 13/100
151/151 - 2s - loss: 5.9975 - 2s/epoch - 10ms/step
Epoch 14/100
151/151 - 2s - loss: 5.9085 - 2s/epoch - 10ms/step
Epoch 15/100
151/151 - 2s - loss: 5.8298 - 2s/epoch - 10ms/step
Epoch 16/100
151/151 - 2s - loss: 5.7441 - 2s/epo

In [90]:
loss, accuracy = model.evaluate(x=predictors, y=label)

151/151 [==============================] - 1s 4ms/step - loss: 0.4021


TypeError: cannot unpack non-iterable float object

In [ ]:
tokenizer.word_index.items()

In [ ]:
mySeedText = 'Finding an Expansive View  of a Forgotten People in Niger'

In [ ]:
token_list = tokenizer.texts_to_sequences([mySeedText])

In [ ]:
token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')


In [79]:
text = tokenizer.sequences_to_texts([[1.4730062e-09 5.9099355e-05 1.2360272e-04 ... 4.9143352e-09
  2.7758798e-03 1.1411373e-06]])


SyntaxError: invalid syntax (Temp/ipykernel_19708/2716408907.py, line 1)

In [84]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)


def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        print("prediction : " ,predicted)
        output_word = ""
        #print("printed word :   ",tokenizer.sequences_to_texts([predicted]))
        text = tokenizer.sequences_to_texts([predicted][0])
    #seed_text += " " + tokenizer.sequences_to_texts([predicted])[0]
    
        for word,index in tokenizer.word_index.items():
    #    if index == predicted:
           output_word = word
    #        break
        seed_text += " "+output_word
    return seed_text.title() #capitalizes the first letter of each word

In [94]:
print(generate_text("united states", 5, model, max_sequence_len))

prediction :  [[7.3804796e-10 1.9010955e-05 7.3539479e-05 ... 4.6765287e-08
  1.0148627e-03 2.3557845e-08]]
prediction :  [[6.0491252e-12 3.6403826e-03 1.5738700e-02 ... 4.0718507e-14
  3.1002265e-13 6.6530686e-17]]
prediction :  [[4.7310107e-11 1.4797232e-02 3.9944630e-02 ... 3.8798115e-10
  1.4318728e-10 6.1417543e-10]]
prediction :  [[5.5074740e-11 1.4792704e-02 2.1227529e-02 ... 8.2328528e-12
  1.3703349e-12 3.4513539e-09]]
prediction :  [[1.13013154e-09 1.33634815e-02 2.22565290e-02 ... 1.49876889e-10
  1.44986079e-10 2.60249653e-04]]
United States Spice Spice Spice Spice Spice


In [95]:
print(generate_text("donald trump", 5, model, max_sequence_len))

prediction :  [[2.22850649e-09 1.67534828e-01 1.21636085e-01 ... 5.54154249e-05
  7.96202073e-08 2.14296603e-09]]
prediction :  [[3.2172437e-10 2.1343797e-02 4.2727750e-02 ... 8.2634838e-10
  8.5084891e-11 8.3246179e-12]]
prediction :  [[4.5430944e-11 2.8389560e-02 9.0045802e-02 ... 8.3979593e-08
  3.2503940e-09 7.6965322e-13]]
prediction :  [[2.7896931e-11 2.2649271e-02 1.4389323e-01 ... 4.1582324e-11
  1.0843644e-09 1.0725366e-13]]
prediction :  [[2.3639719e-11 6.1670482e-02 2.2241782e-01 ... 1.8821757e-11
  2.1140337e-10 1.9538175e-12]]
Donald Trump Spice Spice Spice Spice Spice
